In [1]:
# Setup and Constants
# Defines the base URLs and endpoints for our FastAPI PayPal project
# ------------------------------------------------------------------------------
import requests
import json
import time

BASE_URL = "https://a82e-2001-b011-15-1344-8d58-4497-8d2b-b54d.ngrok-free.app"

LOGIN_URL = BASE_URL + "/api/v1/auth/login"
PRODUCT_URL = BASE_URL + "/api/v1/products"
PLAN_URL = BASE_URL + "/api/v1/plans"
SUBSCRIPTION_URL = BASE_URL + "/api/v1/subscriptions"

In [ ]:
# Login / Get Authorization
# Send a POST request to obtain or verify login credentials
# ------------------------------------------------------------------------------
headers = {
  'Authorization': 'Basic Og==',
  'Content-Type': 'application/json'
}

response = requests.request("POST", LOGIN_URL, headers=headers, data={})
print(response.text)

In [ ]:
# Create a Product
# Demonstrates how to create a new product via POST /products
# ------------------------------------------------------------------------------
payload = json.dumps({
    "name": "Paypal Demo",
    "type": "SERVICE",
    "id": f"adcopy-service-{int(time.time())}",
    "description": "Subscription-based custom ad copy generation powered by ChatGPT.",
    "category": "SOFTWARE",
    "image_url": "https://example.com/gallary/images.jpg",
    "home_url": "https://example.com/catalog.jpg"
})

response = requests.request("POST", PRODUCT_URL, headers=headers, data=payload)
print(json.dumps(response.json(), indent=4))
product_id = response.json().get("id")

In [ ]:
# Retrieve a Specific Product
# Use product_id from the previous cell to fetch that product's details
# ------------------------------------------------------------------------------
response = requests.request("GET", f"{PRODUCT_URL}/{product_id}" , headers=headers, data={})
print(json.dumps(response.json(), indent=4))

In [ ]:
# Create a Plan
# Creates a plan referencing the previously created product
# ------------------------------------------------------------------------------
payload = json.dumps({
    "product_id": product_id,
    "name": "Paypal Demo Plan",
    "description": "Subscription-based custom ad copy generation powered by ChatGPT.",
    "status": "ACTIVE",
    "billing_cycles": [
        {
            "frequency": {
                "interval_unit": "MONTH",
                "interval_count": 1
            },
            "tenure_type": "REGULAR",
            "sequence": 1,
            "total_cycles": 0,
            "pricing_scheme": {
                "fixed_price": {
                    "value": "10000",
                    "currency_code": "JPY"
                }
            }
        }
    ],
    "payment_preferences": {
        "auto_bill_outstanding": True,
        "setup_fee": {
            "value": "1000",
            "currency_code": "JPY"
        },
        "setup_fee_failure_action": "CONTINUE",
        "payment_failure_threshold": 3
    },
    "taxes": {
        "percentage": "10",
        "inclusive": False
    }
})

response = requests.request("POST", PLAN_URL, headers=headers, data=payload)
print(json.dumps(response.json(), indent=4))
plan_id = response.json().get("id")

In [ ]:
# Retrieve a Specific Plan
# Using the plan_id from the previous cell to fetch plan details
# ------------------------------------------------------------------------------
response = requests.request("GET", f"{PLAN_URL}/{plan_id}" , headers=headers, data={})
print(json.dumps(response.json(), indent=4))

In [ ]:
# Create a Subscription
# Creates a subscription linked to the newly created plan
# ------------------------------------------------------------------------------
payload = json.dumps({
    "plan_id": plan_id,
    "shipping_amount": {
        "currency_code": "JPY",
        "value": "0"
    },
    "subscriber": {
        "name": {
            "given_name": "FooBuyer",
            "surname": "Jones"
        },
        "email_address": "foobuyer@example.com",
        "shipping_address": {
            "name": {
                "full_name": "John Doe"
            },
            "address": {
                "address_line_1": "2211 N First Street",
                "address_line_2": "Building 17",
                "admin_area_2": "San Jose",
                "admin_area_1": "CA",
                "postal_code": "95131",
                "country_code": "US"
            }
        }
    },
    "application_context": {
        "brand_name": "Example Inc",
        "locale": "en-US",
        "shipping_preference": "SET_PROVIDED_ADDRESS",
        "user_action": "SUBSCRIBE_NOW",
        "payment_method": {
            "payer_selected": "PAYPAL",
            "payee_preferred": "IMMEDIATE_PAYMENT_REQUIRED"
        },
        "return_url": "https://example.com/return",
        "cancel_url": "https://example.com/cancel"
    }
})

response = requests.request("POST", SUBSCRIPTION_URL, headers=headers, data=payload)
print(json.dumps(response.json(), indent=4))
subscription_id = response.json().get("id")

In [ ]:
# Retrieve a Specific Subscription
# Retrieve the subscription details including plan info by passing 'fields=plan'
# ------------------------------------------------------------------------------
response = requests.request("GET", f"{SUBSCRIPTION_URL}/{subscription_id}" , headers=headers, data={})
print(json.dumps(response.json(), indent=4))

In [ ]:
# User Action: Approve Subscription
# ------------------------------------------------------------------------------
#
# Step 1:
#   Sign in to https://www.sandbox.paypal.com/signin
#   Using the test (personal) account:
#       Email: sb-j4yuh35482103@personal.example.com
#       Password: Th|%Afg0
#
# Step 2:
#   From the "Create a Subscription" API response, locate the approval link.
#   Click this link in your browser to approve the subscription, then you will see "Example Domain" page.
#
# Step 3:
#   After approval, run "Retrieve a Specific Subscription" again.
#   You should now see the subscription status updated to "ACTIVE".
#
# Step 4:
#   Continue with the next steps of your workflow as needed.

In [ ]:
# Patch Subscription
# Updating subscription details, e.g., changing fixed_price
# ------------------------------------------------------------------------------
payload = json.dumps([
    {
        "op": "replace",
        "path": "/plan/billing_cycles/@sequence==1/pricing_scheme/fixed_price",
        "value": {
            "currency_code": "JPY",
            "value": "12345"
        }
    }
])

response = requests.request("PATCH", f"{SUBSCRIPTION_URL}/{subscription_id}", headers=headers, data=payload)
print(json.dumps(response.json(), indent=4))

In [ ]:
# Confirm Subscription Update
# Retrieve the subscription again to confirm the updated pricing scheme
# ------------------------------------------------------------------------------
response = requests.request("GET", f"{SUBSCRIPTION_URL}/{subscription_id}" , headers=headers, params={"fields": "plan"})
print(json.dumps(response.json(), indent=4))

# The "update_time" marks when you performed the "Patch Subscription" action.
# Wait 3-5 minutes for "plan_overridden" to become TRUE and "fixed_price" to update.
# If no changes appear, retry the patch call. (This is managed internally by PayPal.)